In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-27 17:26:03--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-27 17:26:03 (14.0 MB/s) - ‘minsearch.py.2’ saved [3832/3832]



In [1]:
import minsearch

In [3]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-06-25 21:53:24--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json.1’

documents.json.1    100%[===================>] 642.90K  --.-KB/s    in 0.01s   

2024-06-25 21:53:25 (64.2 MB/s) - ‘documents.json.1’ saved [658332/658332]



In [2]:
import json

In [3]:
with open('documents.json', 'rt') as f_in:
    docs_raw= json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course']= course_dict['course']
        documents.append(doc)
        
        

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(text_fields= ['question','text','section'],
               keyword_fields=['course'])

In [7]:
#SELECT * WHERE course = 'data-engineering-zoomcamp';

In [7]:
index.fit(documents)

In [8]:
q= 'the course has already started, can i still join?'

In [9]:
boost= {'question':3.0, 'section': 0.5}

index.search(query=q,
            boost_dict=boost,
            filter_dict= {'course':'data-engineering-zoomcamp'},
            num_results= 5)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineerin

## Modular Functions to Run a LLM (Groq llama3-8b-8192) + RAG code

In [30]:
import os
from groq import Groq

# Set required access to use GROQ_API
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)


In [15]:

def search(query):
    boost = {'question':3.0, 'section': 0.5}

    results = index.search(query = query,
                boost_dict = boost,
                filter_dict = {'course':'data-engineering-zoomcamp'},
                num_results = 5)
    
    return results



def build_prompt(query, search_results):

    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()
    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt

def llm(prompt):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        model="llama3-8b-8192",
    )
    
    return chat_completion.choices[0].message.content

In [41]:
query = 'the course has already started, can I still enroll, the context is availabe try not to slack off?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [42]:
rag(query)

"According to the context, since the course has already started, you can still enroll and join the course. In fact, even if you don't register, you're still eligible to submit the homeworks."